In [ ]:
%matplotlib notebook
import matplotlib.pyplot as plt
import math
import time

from TradingGym.OrderFlow import OrderFlow, readTxt
from TradingGym.OrderBook import OrderBook

In [ ]:
import sys

In [ ]:
flow = OrderFlow()
flow.df =  readTxt('../../Data/Si-3.18/txt/Si-3.18.2017-11-16.OrdLog.{1-OrdLog}.txt')

In [ ]:
book = OrderBook()

In [ ]:
start = 10**4 + 10**3
end = 10**4 + 2 * 10**3
step = 0.1 # in seconds
width = 50 # bp around mid price to show

In [ ]:
def BookIter(end):
    for idx in range(end):
        book.update(flow.df.iloc[idx])
        yield book, flow.df.iloc[idx]

In [ ]:
bookIter = BookIter(end)

In [ ]:
for idx in range(start):
    next(bookIter)

In [ ]:
plt.ion()
fig = plt.figure()
ax = fig.add_subplot(111)
fig.show()
fig.canvas.draw()

In [ ]:
tooSlow = False
for book, message in bookIter:
    if not 'EndOfTransaction' in message.Flags:
        continue
        
    stTime = time.process_time()
    
    midPrice = (max(book.book[0].keys()) + min(book.book[1].keys())) / 2

    ax.clear()
    ax.set_xlim((midPrice - width / 2, midPrice + width / 2))
    ax.set_ylim((0, 8))

    bar1 = ax.bar([x for x in book.book[0].keys()], [math.log(x) for x in book.book[0].values()], color='r')
    bar2 = ax.bar([x for x in book.book[1].keys()], [math.log(x) for x in book.book[1].values()], color='b')
    ax.set_title(str(message.ExchTime))
    
    fig.canvas.draw()
    enTime = time.process_time()
    remaining = step - enTime + stTime
    if (remaining < 0 and not tooSlow):
        print("Warning: the plot is too slow: {} vs. {:.4f}".format(step, enTime - stTime))
        tooSlow = True
    time.sleep(max(0, remaining))